In [1]:
import numpy as np
import pandas as pd

In [2]:
data_dicts = r"/home/ubuntu/PyTorch-YOLOv3/data/custom/images/CAD-120"

In [3]:
import os
anno_folder_list = [name for name in os.listdir(data_dicts) if name.endswith('annotations')]

In [4]:
anno_folder_list

['Subject1_annotations']

All subjects are including ten high-level activities.

So, it do not need the for loop.

In [5]:
high_level_activities_list = os.listdir(data_dicts + '/' + anno_folder_list[0])

In [6]:
high_level_activities_list

['having_meal',
 'unstacking_objects',
 'microwaving_food',
 'taking_medicine',
 'cleaning_objects',
 'taking_food',
 'making_cereal',
 'arranging_objects',
 'picking_objects',
 'stacking_objects']

In [7]:
dict_ten_digit = {} # The final output dictionary

In [8]:
label_paths = {}
for Subject in anno_folder_list:
    for high_level_activity in high_level_activities_list:
        anno_files_folder = data_dicts + '/' + Subject + '/' + high_level_activity
        
        activityLabel = anno_files_folder + '/' + 'activityLabel.txt'
        f = open(activityLabel, 'r')
        my_txt = [txtline[:-1].split(',') for txtline in f.readlines()]
        f.close()
        for _list in my_txt:
            ten_digit = _list[0]
            activity_id = _list[1]
            _img_folder_path = data_dicts + '/' + Subject[:8] + '_rgbd_images' + '/' + high_level_activity + '/' + ten_digit
            dict_ten_digit[ten_digit] = {'img_folder_path' : _img_folder_path, 
                                         'activity_id' : activity_id}
            _label_txt = anno_files_folder + '/' + ten_digit + '.txt'
            try:
                t = open(_label_txt, 'r')
                t.close()
            except:
                continue
            label_paths[ten_digit] = _label_txt

Generate the dictionary which contains all data information

dict_folders = {}#folder_ID:dict_folder

dict_folder = {}#

dict_frames = {}#frame_1:{dict_ROI, dict_P}, frame_2:{dict_ROI, dict_P},...,frame_ID:{dict_ROI, dict_P}

dict_ROI = {}#{1:{orin, CONF},2:{},...,11:{}}

dict_P = {}#{1:{coor, CONF},2:{},3:{},...15:{}}

In [9]:
for i, path in enumerate(label_paths):
    label_file = label_paths[path]
    _txt = open(label_file, 'r')
    my_txt = [txtline[:-1].split(',') for txtline in _txt.readlines()]
    _txt.close()
    dict_frames = {}
    for _frame_data in my_txt[:-1]:
        frame_ID = int(_frame_data.pop(0))
        dict_ROI, dict_P = {}, {}
        for times in range(15):
            if times < 11:
                ORI_i = []
                for t in range(9):
                    ORI_i.append(float(_frame_data.pop(0)))
                ORI_i_CONF = int(_frame_data.pop(0))
                dict_ROI[times] = {'orin':ORI_i, 'CONF':ORI_i_CONF}
            P_i = []
            for t in range(3):
                P_i.append(float(_frame_data.pop(0)))
            P_i_CONF = int(_frame_data.pop(0))
            dict_P[times] = {'coor':P_i, 'CONF':P_i_CONF}         
        dict_frames[frame_ID] = {'ROI':dict_ROI, 'P':dict_P}
    dict_ten_digit[path]['frames'] = dict_frames
    

Get labeling information and put them into **dict_ten_digit** after getting keypoints label

In [ ]:
dict_labeling = {}
for Subject in anno_folder_list:
    for high_level_activity in high_level_activities_list:
        anno_files_folder = data_dicts + '/' + Subject + '/' + high_level_activity
        
        labeling = anno_files_folder + '/' + 'labeling.txt'
        f = open(labeling, 'r')
        my_txt = [txtline[:-1].split(',') for txtline in f.readlines()]
        f.close()
        for _list in my_txt:
            ten_digit = _list[0]
            sub_activity = _list[3]
            inframe = _list[1:2]

Writing to a json file to store the dictionary style data

In [18]:
import json
with open('./data2.json', 'w') as json_file:
    json.dump(dict_ten_digit, json_file, indent=4)
json_file.close()

****************************